# Generate Training Data

We wish to speed up the process of having a human being name entities that they recognize in a paragraph, and having their positions in the paragraph identified and placed in a syntax usable by spaCy's NER training routine.

## Open PDF file, extract page two and display as sentences

In [4]:
import spacy
import PyPDF2
nlp = spacy.load('en_core_web_sm')

#Open PDF file for reading
pdfFile = open("BarCvDescLJ11.pdf", mode="rb")
pdfReader = PyPDF2.PdfFileReader(pdfFile)

# Select a page to work on
pageNumber = 10

# Get text
OnePage = pdfReader.getPage(pageNumber-1) #0-based count
OnePageText = OnePage.extractText()

# Close PDF file
pdfFile.close()

# Remove newlinesxmx82k-&a. It appears multiple newlines together makes
# Spacy think that is the end of a sentence. The PDF reader reads the text in
# an odd fashion
OnePageText = OnePageText.replace('\n','')

# create a spaCy doc object from the page and break it into sentences
doc = nlp(OnePageText)
l=0
for sent in doc.sents:
     print(l, ": ", sent)
     l = l+1


0 :  10  wrinkled.
1 :  Veins are prominent.
2 :  The crease is narrow and shallow at the base and flaring toward the beard end with a well defined fold in the distal half.
3 :  At the time of release, it was susceptible to BYD and moderately resistant to powdery mildew.
4 :  It was evaluated as Entry 655 in the UC Regional Cereal Testing program from 1983-1984 for spring planting in the intermountain region of northern California.
5 :  Crop Science 25:1123 (1985)    
6 :  APEX  Apex is a two-row spring malting and feed barley.
7 :  It was developed by
8 :  Wilbur-Ellis Co. It was selected from the cross Aramir*((Cebaco 6721*(Julia/3/Volla*L100)).
9 :  It is late maturing and is mid-tall with fair straw strength.
10 :  At the time of evaluation it was resistant to leaf rust, net blotch, and powdery mildew.
11 :  It was evaluated as Entry 721 in the UC Regional Cereal Testing program in 1985 and in 1987 for late fall planting in the Central Valley and south-central coastal regions of Ca

## Read in Per-line named entity file and match entities to sentence positions.

In [5]:
import re
import csv
import pandas as pd
fname = "Data/barley_p10_ner.txt"

# Covert the nlp senetence generator into a list of sentences
sentences = list(doc.sents)

# Open the file of manually matched pairs (sentence # <tab> word phrase <tab> named entity)
# e.g.:
#  0      AC Metcalfe     CVAR
#  0      two-rowed       TRAT
#  0      barley          CROP
#  1      Agri-Food Candada   ORG
#  1      1997    DATE
file = open(fname)
reader = csv.reader(file, delimiter='\t', quoting=csv.QUOTE_NONE)
data = list()

for row in reader:
    try:
        (sentIndex, phrase, label) = row
        sent = sentences[int(sentIndex)].string.rstrip()
        
        # find all instances of the 'phrase' in the 'sent'.
        iter = re.finditer(r"\b"+phrase+r"\b", sent)
        indices = [m.start(0) for m in iter]
        
        # check to make sure the phrase the user said was there was indeed found
        if len(indices) == 0:
            raise ValueError
                
        # print out all instances
        for i in indices:
#            print(sentIndex, sent, phrase, "("+str(i), i+len(phrase), "'"+label+"')")
            data.append([sentIndex, sent, phrase, "("+str(i)+", "+str(i+len(phrase))+", '"+label+"')"])
            
    except:
        print("Handle manually: ", row)
        
df = pd.DataFrame(data, columns = ["Index", "Sentence", "Phrase", "MatchInfo"])
print(df)


Handle manually:  ['5', 'Crop Science 25:1123 (1985)', 'JRNL']
Handle manually:  ['8', 'Aramir*((Cebaco 6721*(Julia/3/Volla*L100))', 'PED']
Handle manually:  ['32', 'Crop Science 45: 1160 (2005)', 'JRNL']
Handle manually:  ['49', 'Agronomy Journal 52:661 (1960)', 'JRNL']
Handle manually:  ['53', 'Atlas *3/CIho 3920-1//Atlas 46/3/4* Atlas/CIho 1179//2* Atlas 57', 'PED']
Handle manually:  ['62', 'Crop Science 9(4):521 (1969)', 'JRNL']
    Index                                           Sentence          Phrase  \
0       1                               Veins are prominent.           Veins   
1       2  The crease is narrow and shallow at the base a...          crease   
2       2  The crease is narrow and shallow at the base a...            base   
3       2  The crease is narrow and shallow at the base a...           beard   
4       3  At the time of release, it was susceptible to ...     susceptible   
..    ...                                                ...             ...   
148

## Create a function to clean up overlapping intervals

In [6]:
import re
coordRegex = re.compile(r'(\d+), (\d+)')

def sortByStart(coords):
    """For use in sort routines, return object with lowest (X,Y) values"""
    # split out coordinates that come in as (5, 7, 'CVAR')
    mo = coordRegex.search(coords)
    return(int(mo.group(1)))

def overlaps(coord1, coord2):
    """Check if coordinates of the form 5, 7, 'CVAR' and 32, 46, 'TRAT' overlap"""
    mo1 = coordRegex.search(coord1)
    mo2 = coordRegex.search(coord2)
    coord1Low = int(mo1.group(1))
    coord1High = int(mo1.group(2))
    coord2Low = int(mo2.group(1))
    coord2High = int(mo2.group(2))
    
    if ((coord1High >= coord2Low) and (coord1Low <= coord2Low) or
        (coord2High >= coord1Low) and (coord2Low <= coord1Low)):
        return True
    else:
        return False

def keepFirst(coord1, coord2):
    """Given overlapping coordinates, return the wider encompassing one."""
    mo1 = coordRegex.search(coord1)
    mo2 = coordRegex.search(coord2)
    coord1Low = int(mo1.group(1))
    coord1High = int(mo1.group(2))
    coord2Low = int(mo2.group(1))
    coord2High = int(mo2.group(2))
 
    if (int(coord1High) - int(coord1Low)) >= (int(coord2High) - int(coord2Low)):
        return True
    else:
        return False

# print("Should be false:", overlaps("(5, 7, 'CVAR')", "(32, 46, 'TRAT')"))
# print("Should be true:", overlaps("(26, 46, 'TRAT')", "(32, 46, 'TRAT')"))
# print("Should be true:", overlaps("(26, 46, 'TRAT')", "(26, 46, 'TRAT')"))
# print("Keeper:", keepFirst("34, 46, 'TRAT'", "34, 46, 'TRAT'"))

def cleanIntervals(inputString=""):
    """order intervals like (5, 7, 'CVAR'), (32, 46, 'TRAT'), (26, 46, 'TRAT') and remove overlapping ones."""
    inputString = inputString.lstrip("(").rstrip(")")
    intervalList = inputString.split("), (")
    intervalList.sort(key = sortByStart)
#    print("Sorted Interval List:", intervalList)

    # Pairwise compare every interval in the list to every other interval to check overlap
    keeperList = [True]*len(intervalList) # Logic array to determine if each interval should be kept
    i=0
    for interval1 in intervalList:
        for interval2 in intervalList:
            if interval1 == interval2:
                if intervalList.index(interval1) != i: # when both are the same we reject the higher one
                    keeperList[i] = False
            else:
                if overlaps(interval1, interval2) and keepFirst(interval1, interval2) == False:
                    keeperList[i] = False
        i = i+1
        
#    print("keeperList:", keeperList)
   
    # Build up the return interval list
    returnStr = "("
    for interval, isKeeper in zip(intervalList, keeperList):
        if isKeeper:
            returnStr = returnStr + interval + "), ("
    return (returnStr.rstrip("), (") + ")")
        
# cleanIntervals("(5, 7, 'CVAR'), (32, 46, 'TRAT'), (5, 9, 'CVAR'), (48, 55, 'ORG'), (26, 46, 'TRAT')")
# cleanIntervals("(0, 8, 'CVAR'), (0, 5, 'CVAR'), (21, 26, 'PLAN'), (32, 37, 'CVAR')")
cleanIntervals("(0, 12, 'CVAR'), (39, 49, 'CVAR'), (39, 49, 'CVAR'), (71, 77, 'CVAR'), (71, 77, 'CVAR'), (92, 113, 'TRAT'), (140, 150, 'CVAR'), (140, 150, 'CVAR'), (181, 187, 'CVAR'), (181, 187, 'CVAR')")


"(0, 12, 'CVAR'), (39, 49, 'CVAR'), (71, 77, 'CVAR'), (92, 113, 'TRAT'), (140, 150, 'CVAR'), (181, 187, 'CVAR')"

## Aggregate all matches for each sentence on a single line and output in spaCy training format

In [7]:
# use Pandas dataframes to aggregate all entity matches together for a single sentence
agg_rules = {'Sentence': 'first', 'Phrase': 'first', 'MatchInfo': lambda x: ', '.join(x)}
res = df.groupby('Index').agg(agg_rules)
#print(res)

# Now format it just like what is needed for the spaCy training module: 
# E.g.:
# ('Eight-Twelve is a six-rowed winter feed barley', {'entities': [(0, 12, 'CVAR'), (18, 27, 'TRAT'), (28, 39, 'TRAT'),(40, 46, 'CROP')]}),
records = res.to_dict('records')
for i in range(0,len(records)):
    print("    ('"+records[i]['Sentence']+"', {'entities': ["+cleanIntervals(records[i]['MatchInfo'])+"]}),")
    #print("    ('"+records[i]['Sentence']+"', {'entities': ["+records[i]['MatchInfo']+"]}),")


    ('Veins are prominent.', {'entities': [(0, 5, 'PLAN')]}),
    ('At the time of evaluation it was resistant to leaf rust, net blotch, and powdery mildew.', {'entities': [(33, 42, 'PPTD'), (46, 55, 'PATH'), (57, 67, 'PATH'), (73, 87, 'PATH')]}),
    ('AQUILA  Aquila is a six-row spring feed barley.', {'entities': [(8, 14, 'CVAR'), (20, 27, 'TRAT'), (28, 34, 'TRAT'), (35, 39, 'TRAT'), (40, 46, 'CROP')]}),
    ('It was released by the Utah AES in 2003.', {'entities': [(23, 31, 'ORG'), (35, 39, 'DATE')]}),
    ('It was selected from the cross UT-S.D. B1-1009/M72-395/3/Utah Short #2//ID633019/Woodvale/4/Steptoe/M27//Westbred Gustoe.', {'entities': [(31, 120, 'PED')]}),
    ('Its experimental designation was UT97B1480-1632.', {'entities': [(33, 47, 'ALAS')]}),
    ('It is early heading and has very good lodging resistance.', {'entities': [(12, 19, 'TRAT'), (38, 56, 'TRAT')]}),
    ('Plant height is mid-tall, similar to Steptoe.', {'entities': [(0, 12, 'TRAT'), (16, 24, 'TRAT'), (37, 44, '